# CBOW model for text classification
We develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

In [1]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [2]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [3]:
unpack_dataset()

--2024-02-25 11:24:58--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz’

rotten_imdb.tar.gz  100%[===================>] 507.42K  --.-KB/s    in 0.002s  

2024-02-25 11:24:59 (212 MB/s) - ‘rotten_imdb.tar.gz’ saved [519599/519599]

x quote.tok.gt9.5000
x plot.tok.gt9.5000
x subjdata.README.1.0


In [4]:
!ls data

plot.tok.gt9.5000   quote.tok.gt9.5000  subjdata.README.1.0


In [5]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [6]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [7]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [8]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [9]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [10]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Better tokenization with Spacy

In [13]:
#!pip install -U spacy

In [11]:
import spacy

In [25]:
# first time run this
#!python3 -m spacy download en_core_web_sm

In [12]:
tok = spacy.load('en_core_web_sm')

In [13]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [14]:
len(obj_lines)

5000

In [15]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [16]:
test = tok(obj_lines[0])

In [17]:
np.array([x for x in test])

array([the, movie, begins, in, the, past, where, a, young, boy, named,
       sam, attempts, to, save, celebi, from, a, hunter, ., 
], dtype=object)

## Split dataset in train and validation

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [20]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'),
 array([1., 0., 0., 1., 1.]))

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [23]:
from collections import defaultdict

In [24]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [25]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [26]:
#word_count

In [27]:
len(word_count.keys())

21415

In [28]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [29]:
len(word_count.keys())

4065

In [30]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [31]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [32]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_val_len = np.array([len(x.split()) for x in X_val])

In [33]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

43.0

In [34]:
X_train[0]

'will god let her fall or give her a new path ?'

In [35]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

11

In [36]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 4,  9,  8,  5,  2,  3,  7,  5, 12, 10,  6, 11])

In [37]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [38]:
encode_sentence(X_train[0])

array([ 4,  9,  8,  5,  2,  3,  7,  5, 12, 10,  6, 11,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [39]:
x_train_len = np.minimum(x_train_len, 40)
x_val_len = np.minimum(x_val_len, 40)

In [40]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [41]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [42]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.7863,  0.6172,  0.7054, -0.1227],
        [-0.2579, -0.8458,  0.5222,  1.9155],
        [ 0.9924, -1.1376,  0.1226, -0.9755],
        [-0.1674,  0.6059,  0.9980,  0.7524],
        [ 0.1461,  0.9249, -0.3235, -0.3222],
        [-1.7929,  0.0973,  0.0292, -0.9028],
        [ 0.7267,  0.7262, -1.4612, -0.1349],
        [-1.0735,  0.2862, -1.2463,  0.7119],
        [-0.1202, -0.2927,  0.7785,  0.6602]], requires_grad=True)

Note that the `padding_idx` has embedding vector 0.

In [43]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[ 0.7863,  0.6172,  0.7054, -0.1227],
         [-0.1674,  0.6059,  0.9980,  0.7524],
         [ 0.7863,  0.6172,  0.7054, -0.1227],
         [ 0.1461,  0.9249, -0.3235, -0.3222],
         [ 0.7863,  0.6172,  0.7054, -0.1227],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [44]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [45]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [46]:
embed(a).shape

torch.Size([2, 3, 4])

In [47]:
embed(a)

tensor([[[ 0.7863,  0.6172,  0.7054, -0.1227],
         [-0.1674,  0.6059,  0.9980,  0.7524],
         [ 0.7863,  0.6172,  0.7054, -0.1227]],

        [[ 0.7863,  0.6172,  0.7054, -0.1227],
         [ 0.9924, -1.1376,  0.1226, -0.9755],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)

In [48]:
embed(a).sum(dim=1)

tensor([[ 1.4051,  1.8404,  2.4089,  0.5070],
        [ 1.7786, -0.5204,  0.8280, -1.0982]], grad_fn=<SumBackward1>)

In [49]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[ 0.4684,  0.6135,  0.8030,  0.1690],
        [ 0.8893, -0.2602,  0.4140, -0.5491]], grad_fn=<DivBackward0>)

## Continuous Bag of Words Model

In [50]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

In [51]:
model = CBOW(vocab_size=5, emb_size=3)

In [52]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.2858,  0.6973,  2.1065],
        [-0.3293,  0.4933, -0.6576],
        [ 1.7065,  1.0676,  0.4678],
        [-0.7916, -0.6245, -1.1746]], requires_grad=True)

In [53]:
s = s.view(s.shape[0], 1)
model(a, s)

tensor([[-0.7284],
        [-0.8112]], grad_fn=<AddmmBackward0>)

# Training the CBOW model 

In [54]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [55]:
def val_metrics(model):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    s = torch.Tensor(x_val_len).view(x_val_len.shape[0], 1)
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [56]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.7012071013450623, 0.46050000190734863)

In [57]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        s = torch.Tensor(x_train_len).view(x_train_len.shape[0], 1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, val_accuracy = val_metrics(model)
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % 
              (loss.item(), val_loss, val_accuracy))

In [58]:
model = CBOW(vocab_size=V, emb_size=50)
train_epocs(model, epochs=10, lr=0.1)

train_loss 0.702 val_loss 0.718 val_accuracy 0.508
train_loss 0.701 val_loss 0.609 val_accuracy 0.703
train_loss 0.596 val_loss 0.522 val_accuracy 0.857
train_loss 0.506 val_loss 0.431 val_accuracy 0.872
train_loss 0.405 val_loss 0.344 val_accuracy 0.887
train_loss 0.307 val_loss 0.290 val_accuracy 0.891
train_loss 0.240 val_loss 0.258 val_accuracy 0.900
train_loss 0.193 val_loss 0.247 val_accuracy 0.905
train_loss 0.165 val_loss 0.250 val_accuracy 0.904
train_loss 0.145 val_loss 0.257 val_accuracy 0.910


In [59]:
train_epocs(model, epochs=10, lr=0.01)

train_loss 0.126 val_loss 0.257 val_accuracy 0.910
train_loss 0.122 val_loss 0.256 val_accuracy 0.910
train_loss 0.118 val_loss 0.255 val_accuracy 0.911
train_loss 0.115 val_loss 0.254 val_accuracy 0.911
train_loss 0.112 val_loss 0.254 val_accuracy 0.909
train_loss 0.108 val_loss 0.255 val_accuracy 0.909
train_loss 0.105 val_loss 0.255 val_accuracy 0.909
train_loss 0.102 val_loss 0.255 val_accuracy 0.909
train_loss 0.099 val_loss 0.256 val_accuracy 0.908
train_loss 0.096 val_loss 0.257 val_accuracy 0.909


# Data loaders for SGD

Nearly all of deep learning is powered by one very important algorithm: **stochastic gradient descent (SGD)**. SGD can be seeing as an approximation of **gradient descent** (GD). In GD you have to run through *all* the samples in your training set to do a single itaration. In SGD you use *only one* or *a subset*  of training samples to do the update for a parameter in a particular iteration. The subset use in every iteration is called a **batch** or **minibatch**.

In [60]:
from torch.utils.data import Dataset, DataLoader

Next we are going to create a data loader. The data loader provides the following features:
* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

In [61]:
def encode_sentence2(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [62]:
encode_sentence2(X_train[0])

(array([ 4,  9,  8,  5,  2,  3,  7,  5, 12, 10,  6, 11,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32),
 12)

In [63]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence2(x)
        return x, self.y[idx], s
    
sub_dataset_train = SubjectivityDataset(X_train, y_train)

In [64]:
train_loader = DataLoader(sub_dataset_train, batch_size=5, shuffle=True)
x, y, s = next(iter(train_loader))

In [65]:
x, y, s

(tensor([[  57,    1,   57, 1112,   19,  656,  711,   51,   19,  157,   27,   45,
            12,  942,   14,    1,  222,   39,   10, 2340,    1,    1,  251,   27,
           469,    1, 1635,  474,  602,   86,   12,    1,   14,    1,   76,   12,
             1,   86, 1657, 1351],
         [  12, 1771,   27,    1,   14,    1,  232,  203,  182,   94,   27,   29,
            12, 1771,  232,  182,  948,   32,   94,   12,    1,  188,  115,   12,
          1257, 2947,   27,   14, 2068,   16,   16,   16,    0,    0,    0,    0,
             0,    0,    0,    0],
         [  12,    1,  977,   98, 1420,   41,  168,   32,    1,   19, 2058,   16,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [3976,   27,    1,  804,    1,  132, 2457,  132,    1,   16,  839,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

In [66]:
model = CBOW(vocab_size=V, emb_size=50)

In [67]:
train_loader = DataLoader(sub_dataset_train, batch_size=500, shuffle=True)

In [68]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        total_loss = 0
        total = 0
        model.train()
        for x, y, s in train_loader:
            x = x.type(torch.LongTensor)  #.cuda()
            y = y.type(torch.FloatTensor).unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x, s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
        train_loss = total_loss/total
        val_loss, val_accuracy = val_metrics(model)
        
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (train_loss, val_loss, val_accuracy))

In [69]:
train_epocs(model, epochs=10)

train_loss 0.653 val_loss 0.597 val_accuracy 0.766
train_loss 0.511 val_loss 0.438 val_accuracy 0.848
train_loss 0.346 val_loss 0.317 val_accuracy 0.884
train_loss 0.241 val_loss 0.263 val_accuracy 0.894
train_loss 0.186 val_loss 0.242 val_accuracy 0.902
train_loss 0.151 val_loss 0.236 val_accuracy 0.905
train_loss 0.128 val_loss 0.234 val_accuracy 0.910
train_loss 0.109 val_loss 0.237 val_accuracy 0.908
train_loss 0.094 val_loss 0.242 val_accuracy 0.906
train_loss 0.082 val_loss 0.249 val_accuracy 0.905


## Lab: 

Create a dataset and a model for the Quora dataset. The objective is to predict whether two given sentences share the same intent. Utilize a Siamese-type model where the sentences are individually encoded by the Continuous Bag of Words (CBOW) model, resulting in vector embeddings $a_1$ and $a_2$ for each sentence. Perform a linear transformation on the vector $a_1-a_2$. The final prediction will involve applying a sigmoid function to the output of the linear transformation: $\hat{y} = \sigma(linear(a_1-a_2))$.